I wanted to build a scraper that would allow me to easily pull down NYPD's quarterly data about drone flights. To do so, first I had to import libraries to prepare to scrape the website. Scraping the website itself would be fairly straightforward as the urls followed a simple pattern. Base_url + {year}-{quarter}, which automatically downloads the excel file. 

In [16]:
## import libraries
import requests  # Makes HTTP requests to fetch web pages from URLs
from bs4 import BeautifulSoup  # Parses HTML content into navigable Python objects for web scraping
import pandas as pd  # Creates and manipulates DataFrames for organizing scraped data into tables
import time  # Adds delays between requests to avoid overwhelming the server
from random import uniform  # Generates random time intervals to make scraping delays less predictable
import wget ## allows to use to fetch the links from the website

In [18]:
## create headers to be sneaky

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

In [20]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


The NYPD data seemed clean on first glace... but it turned out I needed to create function to allow to me skip rows in a "smart" way because not every excel file was formated the exact same way. 

My intial code I wrote by hand worked perfectly up until q4 2020 when the row/columns get messed up. That's when I turned to AI to help me write the function -- I had it pivot on "Brooklyn" because that value stays the same throughout. The number of rows/variables horizontally stays the same but the # of columns changes as new types of drone flights are introduced over time.

## here's how I tested it out on my problem sheet 

def get_num_rows_to_skip(raw_df):
    """
    Find the row containing 'Brooklyn' as the header row.
    Skip up to that row.
    """
    for i, row in raw_df.iterrows():
        # Normalize all cells: convert to string, strip whitespace, drop empty cells
        cleaned = row.astype(str).str.strip().fillna("")
        if cleaned.str.contains(r"\bBrooklyn\b", case=False, regex=True).any():
            return i
    return 0  # fallback


def get_column_range_to_use(raw_df):
    """
    Determine column range based on whether 'Other Agency' column exists.
    If column I is missing or fully empty, use A:H; else use A, D:J
    """
    header_row_idx = get_num_rows_to_skip(raw_df)
    header = raw_df.iloc[header_row_idx]

    # Count actual non-null columns
    num_cols = header.count()

    # Case 1: If sheet only has 8 columns → use A:H**
    if num_cols <= 8:
        return "A:H"

    # Case 2: If column I (index 8) exists but is fully empty → use A:H
    if header.iloc[8] in [None, "", "nan"] or pd.isna(header.iloc[8]):
        return "A:H"

    # Otherwise sheet has valid 'Other Agency' column → use extended range
    return "A, D:J"

# download and inspect excel file
url = "https://www.nyc.gov/assets/nypd/downloads/excel/analysis_and_planning/uas/uas-operations-2020-q4.xlsx"
my_data = wget.download(url)

# smart function 
num_rows_to_skip = get_num_rows_to_skip(raw_df)
col_range_to_use = get_column_range_to_use(raw_df)
df = pd.read_excel(my_data, skiprows=num_rows_to_skip, usecols=col_range_to_use)

# Now you can rename the first column
df.rename(columns={df.columns[0]: "Category"}, inplace=True)

df.tail()

After I knew that function worked I integrated into the code I had built by hand earlier. 

In [26]:
all_dfs = []  ## hold all dfs (dataframes) that will be created for each page
base_url = "https://www.nyc.gov/assets/nypd/downloads/excel/analysis_and_planning/uas/uas-operations-" ## base url of the nypd site which holds the files

## smart row and column function 

def get_num_rows_to_skip(raw_df):
    """
    Find the row containing 'Brooklyn' as the header row.
    Skip up to that row.
    """
    for i, row in raw_df.iterrows():
        # Normalize all cells: convert to string, strip whitespace, drop empty cells
        cleaned = row.astype(str).str.strip().fillna("")
        if cleaned.str.contains(r"\bBrooklyn\b", case=False, regex=True).any():
            return i
    return 0  # fallback


def get_column_range_to_use(raw_df):
    """
    Determine column range based on whether 'Other Agency' column exists.
    If column I is missing or fully empty, use A:H; else use A, D:J
    """
    header_row_idx = get_num_rows_to_skip(raw_df)
    header = raw_df.iloc[header_row_idx]

    # Count actual non-null columns
    num_cols = header.count()

    # Case 1: If sheet only has 8 columns → use A:H**
    if num_cols <= 8:
        return "A:H"

    # Case 2: If column I (index 8) exists but is fully empty → use A:H
    if header.iloc[8] in [None, "", "nan"] or pd.isna(header.iloc[8]):
        return "A:H"

    # Otherwise sheet has valid 'Other Agency' column → use extended range
    return "A, D:J"

## main scraping loop 

for year in range(2019, 2026):
    for quarter in range (1, 5): 
        try:  ## attempt to request the page
            url = f"{base_url}{year}-q{quarter}.xlsx"
            current_page_data = wget.download(url)

            ## use smart row and column function 
            raw_df = pd.read_excel(current_page_data, header=None)
            num_rows_to_skip = get_num_rows_to_skip(raw_df)
            col_range_to_use = get_column_range_to_use(raw_df)

            ## read data 
            df = pd.read_excel(current_page_data, skiprows=num_rows_to_skip, usecols=col_range_to_use)

            # rename first column
            df.rename(columns={df.columns[0]: "Category"}, inplace=True)
            
            # add metadata (year/quarter)
            df["Year"] = year
            df["Quarter"] = f"Q{quarter}"

            # Add this DataFrame to the list
            all_dfs.append(df)
            
            ## pause between page requests to avoid overwhelming the server (random delay between 30–45 seconds)
            second_to_snooze = uniform(30,45)
            print(f"Created DF from page {url} and snoozing for {second_to_snooze} seconds before next page")
            time.sleep(second_to_snooze)  ## actually wait the random time before continuing
        
        except:  ## if something broke somewhere in here
            print(f"Problem with {year}-q{quarter}")
            
print(f"Done scraping all available quarters of drone data")  ## confirm completion once all quarters are processed

100% [..........................................................] 14128 / 14128Created DF from page https://www.nyc.gov/assets/nypd/downloads/excel/analysis_and_planning/uas/uas-operations-2019-q1.xlsx and snoozing for 36.52790578344512 seconds before next page
100% [..........................................................] 14142 / 14142Created DF from page https://www.nyc.gov/assets/nypd/downloads/excel/analysis_and_planning/uas/uas-operations-2019-q2.xlsx and snoozing for 44.75741930037232 seconds before next page
100% [..........................................................] 15376 / 15376Created DF from page https://www.nyc.gov/assets/nypd/downloads/excel/analysis_and_planning/uas/uas-operations-2019-q3.xlsx and snoozing for 30.955305516157946 seconds before next page
100% [..........................................................] 17169 / 17169Created DF from page https://www.nyc.gov/assets/nypd/downloads/excel/analysis_and_planning/uas/uas-operations-2019-q4.xlsx and snoozin

In [28]:
## put them all together

df = pd.concat(all_dfs, ignore_index = True)
df

,Category,Unnamed: 1,Unnamed: 2,Brooklyn,Bronx,Queens,Manhattan,Staten Island,Year,Quarter,Other Agency,Total,Other Agency\nAssist
0,Search and rescue operations,NaN,NaN,0,0,0,0,0,2019,Q1,NaN,NaN,NaN
1,Collision / Crime Scene Documentation,NaN,NaN,1,1,4,0,0,2019,Q1,NaN,NaN,NaN
2,Evidence searches at large or inaccessible scenes,NaN,NaN,0,1,0,0,0,2019,Q1,NaN,NaN,NaN
3,Hazardous material incidents,NaN,NaN,0,0,0,0,0,2019,Q1,NaN,NaN,NaN
4,Monitoring vehicular traffic and pedestrian co...,NaN,NaN,0,0,0,2,0,2019,Q1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,"Public safety, emergency, or other situation w...",NaN,NaN,30,79,82,29,111,2025,Q1,NaN,NaN,NaN
293,Training /Testing,NaN,NaN,13,4,9,3,0,2025,Q1,NaN,NaN,NaN
294,Drone as a First Responder \n(DFR),NaN,NaN,1215,988,0,202,3,2025,Q1,NaN,NaN,NaN
295,Warrant,NaN,NaN,24,14,32,2,0,2025,Q1,NaN,NaN,NaN


In [65]:
## how many DFR flight have occured since the policy has been passed

df[df["Category"] == "Drone as a First Responder \n(DFR)"][["Brooklyn","Bronx","Queens","Manhattan","Staten Island","Other Agency", "Other Agency\nAssist"]].sum()

Brooklyn                3053.0
Bronx                   1849.0
Queens                     2.0
Manhattan                310.0
Staten Island              5.0
Other Agency               0.0
Other Agency\nAssist       0.0
dtype: float64

In [67]:
## how many total drone flights across the boros

df[df["Category"] == "TOTAL"][["Brooklyn","Bronx","Queens","Manhattan","Staten Island","Other Agency", "Other Agency\nAssist"]].sum()

Brooklyn                3701.0
Bronx                   2325.0
Queens                  1051.0
Manhattan                999.0
Staten Island            296.0
Other Agency               0.0
Other Agency\nAssist       0.0
dtype: float64

From here I can clean and refine data... 